In [ ]:
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, roc_curve, auc
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LSTM
from tensorflow.keras.optimizers import Adam
1️⃣ Load Landslide Sample Points and Rasters
python
Copy code
# Landslide sample points
points = gpd.read_file("data/samples/landslide_points.shp")
labels = points["label"].values

# 22 conditioning factors including PS-InSAR deformation
features = [
    'grid_code','curvclass','Elevclass','Ndwiclass','Aspectclas',
    'Disfaultcl','Disroadcla','Drainclass','Lulcclass','NDVIclass',
    'Pgaclass','Popclass','Rainclass','Rdlsclass','Relclass1',
    'Slopeclass','Spiclass','Sticlass','tpiclass','Triclass',
    'Twiclass','Disrivercl'
]

# Map raster paths
raster_files = {feat: f"data/rasters/{feat}.tif" for feat in features}
rasters = {name: rasterio.open(path) for name, path in raster_files.items()}

# Ensure CRS match
points = points.to_crs(rasters[features[0]].crs)
coords = [(geom.x, geom.y) for geom in points.geometry]

# Extract raster values at points
X = []
for feat in features:
    raster = rasters[feat]
    vals = np.array([v[0] for v in raster.sample(coords)])
    X.append(vals)

X = np.vstack(X).T
df = pd.DataFrame(X, columns=features)
df["label"] = labels
df = df.dropna()

X = df.drop(columns=["label"]).values
y = df["label"].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
2️⃣ Deep Learning Models
python
Copy code
def build_dnn(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.1),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_bpnn(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_rnn(input_dim):
    model = Sequential([
        SimpleRNN(128, return_sequences=True, input_shape=(input_dim,1)),
        Dropout(0.3),
        SimpleRNN(64),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_lstm(input_dim):
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(input_dim,1)),
        Dropout(0.4),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model
3️⃣ Hybrid Ensemble Machine Learning
python
Copy code
# Voting ensemble
voting_model = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=1000)),
        ('knn', KNeighborsClassifier(n_neighbors=3)),
        ('svc', SVC(probability=True))
    ], voting='soft'
)

# Stacking ensemble
stacking_model = StackingClassifier(
    estimators=[
        ('dt', DecisionTreeClassifier()),
        ('gnb', GaussianNB()),
        ('svc', SVC(probability=True))
    ],
    final_estimator=RandomForestClassifier(n_estimators=200),
    cv=5
)

# Hybrid Boosting
xgb_model = XGBClassifier(n_estimators=150, learning_rate=0.05, max_depth=4)
ada_model = RandomForestClassifier(n_estimators=200, max_depth=10)

# Meta-learner
meta_model = StackingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=50, max_depth=5)),
        ('lgb', LGBMClassifier(n_estimators=50)),
    ],
    final_estimator=LogisticRegression(),
    cv=5
)
4️⃣ SHAP Feature Importance
python
Copy code
# Example: using Random Forest for SHAP
explainer = shap.TreeExplainer(ada_model)
shap_values = explainer.shap_values(X_scaled)

# Summary plot
shap.summary_plot(shap_values[1], X_scaled, feature_names=features)
5️⃣ 10-Fold Stratified CV with ROC–AUC
python
Copy code
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

metrics = {"Accuracy": [], "Precision": [], "Recall": [], "F1": [], "Kappa": [], "ROC-AUC": []}

# Print results
for k,v in metrics.items():
    print(f"{k}: {np.mean(v):.3f}")
 7. Print Results Table
print(f"{'Model':<15} {'Acc':>6} {'Prec':>6} {'Rec':>6} {'F1':>6} {'Kappa':>6} {'AUC':>6}")
print("="*60)
for name, (acc, prec, rec, f1, kappa, auc_score) in results.items():
    print(f"{name:<15} {acc:6.3f} {prec:6.3f} {rec:6.3f} {f1:6.3f} {kappa:6.3f} {auc_score:6.3f}")

# 8. ROC Plot
plt.figure(figsize=(10, 8))
for name, (fpr, tpr, auc_score) in roc_data.items():
    plt.plot(fpr, tpr, label=f"{name} (AUC = {auc_score:.3f})", linewidth=2)
# Diagonal baseline
plt.plot([0, 1], [0, 1], 'k--', linewidth=2)

plt.xlabel('1−Specificity', fontsize=30)
plt.ylabel('Sensitivity', fontsize=30)
plt.title('(a) Training dataset', fontsize=30, fontweight='bold')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(loc='lower right', fontsize=18)
plt.grid(False)
# Enhanced axis border
ax = plt.gca()
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_linewidth(2)
    spine.set_color('black')

plt.show()   
    
6️⃣ Generate Final Raster-Based Susceptibility Map with PS-InSAR Correction
python
Copy code
template = rasters[features[0]]
profile = template.profile
profile.update(dtype=rasterio.float32, count=1)

# Stack all rasters for prediction
stack = np.stack([r.read(1) for r in rasters.values()])
flat_stack = stack.reshape(stack.shape[0], -1).T
flat_scaled = scaler.transform(flat_stack)

# Predict susceptibility
sus_prob = model.predict_proba(flat_scaled)[:,1]

# PS-InSAR correction: increase susceptibility where Vslope is high
vslope_idx = features.index('Disrivercl')  # replace with Vslope raster index if needed
vslope_flat = flat_stack[:, vslope_idx]
sus_prob[vslope_flat>0.005] += 0.1  # example: increase by 10%
sus_prob = np.clip(sus_prob, 0, 1)

sus_map = sus_prob.reshape(template.shape)
with rasterio.open("lsm_corrected.tif", "w", **profile) as dst:
    dst.write(sus_map,1)

print("Final PS-InSAR corrected Landslide Susceptibility Map saved as 'lsm_corrected.tif'")